<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>

# Train SAR on MovieLens with Azure Machine Learning (Python, CPU)

This notebook provides an exmaple of how to train SAR on remote compute resources. Details of SAR algorithm can be found in [SAR Python CPU Movielens](https://github.com/Microsoft/Recommenders/blob/master/notebooks/00_quick_start/sar_movielens.ipynb) notebook. 

This notebook provides an example of how to utilize and evaluate SAR in Python on a CPU.

This notebook showcases what are the changes required to run local script on AzureML targets. AzureML comes with auto-scaling and gpu options and can greatly accelerate model training. There're a few more advanced AzureML notebooks in this repo. 

SAR is a fast scalable adaptive algorithm for personalized recommendations based on user transaction history. It produces easily explainable / interpretable recommendations and handles "cold item" and "semi-cold user" scenarios. SAR is a kind of neighborhood based algorithm (as discussed in [Recommender Systems by Aggarwal](https://dl.acm.org/citation.cfm?id=2931100)) which is intended for ranking top items for each user. 

SAR recommends items that are most ***similar*** to the ones that the user already has an existing ***affinity*** for. Two items are ***similar*** if the users who have interacted with one item are also likely to have interacted with another. A user has an ***affinity*** to an item if they have interacted with it in the past.

### Advantages of SAR:
- High accuracy for an easy to train and deploy algorithm
- Fast training, only requiring simple counting to construct matrices used at prediction time. 
- Fast scoring, only involving multiplication of the similarity matric with an affinity vector

### Advantages of using AML:
- Run large dataset quickly with scaling capability

### Notes to use SAR properly:
- Since it does not use item or user features, it can be at a disadvantage against algorithms that do.
- It's memory-hungry, requiring the creation of an $mxm$ sparse square matrix (where $m$ is the number of items). This can also be a problem for many matrix factorization algorithms.
- SAR favors an implicit rating scenario and it does not predict ratings.

# 0 Setup environment
### 1 Configure AzureML workspace and create experiment
**AzureML workspace** is a foundational block in the cloud that you use to experiment, train, and deploy machine learning models via AzureML service. In this notebook, we 1) create a workspace from [**Azure portal**](https://portal.azure.com) and 2) configure from this notebook.

You can find more details about the setup and configure processes from the following links:
* [Quickstart with Azure portal](https://docs.microsoft.com/en-us/azure/machine-learning/service/quickstart-get-started)
* [Quickstart with Python SDK](https://docs.microsoft.com/en-us/azure/machine-learning/service/quickstart-create-workspace-with-python)

#### 1.1 Create a workspace
1. Sign in to the [Azure portal](https://portal.azure.com) by using the credentials for the Azure subscription you use.
2. Select **Create a resource** menu, search for **Machine Learning service workspace** select **Create** button.
3. In the **ML service workspace** pane, configure your workspace with entering the *workspace name* and *resource group* (or **create new** resource group if you don't have one already), and select **Create**. It can take a few moments to create the workspace.

#### 1.2 Configure
To configure this notebook to communicate with the workspace, type in your Azure subscription id, the resource group name and workspace name to `<subscription-id>`, `<resource-group>`, `<workspace-name>` in the above notebook cell. Alternatively, you can create a *.\\aml_config\\config.json* file with the following contents:
```
{
    "subscription_id": "<subscription-id>",
    "resource_group": "<resource-group>",
    "workspace_name": "<workspace-name>"
}
```

In [1]:
# set the environment path to find Recommenders
import sys
sys.path.append("../../")
import os
import shutil

from reco_utils.dataset import movielens

import azureml
from azureml.core import Workspace, Run, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.train.estimator import Estimator
from azureml.widgets import RunDetails

# AzureML workspace info. Note, will look up "aml_config\config.json" first, then fall back to use this
subscription_id = os.getenv("SUBSCRIPTION_ID", default="<my-subscription-id>")
resource_group = os.getenv("RESOURCE_GROUP", default="<my-resource-group>")
workspace_name = os.getenv("WORKSPACE_NAME", default="<my-workspace-name>")
workspace_region = os.getenv("WORKSPACE_REGION", default="eastus2")

# Remote compute (cluster) configuration. If you want to save the cost more, set these to small.
VM_SIZE = 'STANDARD_D2_V2'
VM_PRIORITY = 'lowpriority'
# Cluster nodes
MIN_NODES = 4
MAX_NODES = 8

Now let's see if everything is ready!

In [2]:
# Connect to a workspace
try:
    ws = Workspace.from_config()
except aml.exceptions.UserErrorException:
    try:
        ws = Workspace(
            subscription_id=subscription_id,
            resource_group=resource_group,
            workspace_name=workspace_name
        )
        ws.write_config()
    except:
        ws = None

if ws is None:
    raise ValueError(
        """Cannot access the AzureML workspace w/ the config info provided.
        Please check if you entered the correct id, group name and workspace name"""
    )
else:
    print("AzureML workspace name: ", ws.name)

Found the config file in: /data/home/cathack/notebooks/Recommenders/notebooks/aml_config/config.json
AzureML workspace name:  setup-ws


### 2 Create Remote Compute Target
We create a cpu cluster as our **remote compute target**. If a cluster with the same name is already exist in your workspace, the script will load it instead. You can see [this document](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-set-up-training-targets) to learn more about setting up a compute target on different locations.

This notebook selects 'STANDARD_NC6' virtual machine (VM) and sets it's priority as lowpriority to save the cost.

In [3]:
CLUSTER_NAME = 'cpucluster'

try:
    compute_target = ComputeTarget(workspace=ws, name=CLUSTER_NAME)
    print("Found existing compute target")
except:
    print("Creating a new compute target...")
    compute_config = AmlCompute.provisioning_configuration(
        vm_size=VM_SIZE,
        vm_priority=VM_PRIORITY,
        min_nodes=MIN_NODES,
        max_nodes=MAX_NODES
    )
    # create the cluster
    compute_target = ComputeTarget.create(ws, CLUSTER_NAME, compute_config)
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

# Use the 'status' property to get a detailed status for the current cluster. 
print(compute_target.status.serialize())

Found existing compute target
{'allocationState': 'Steady', 'allocationStateTransitionTime': '2019-03-21T22:51:59.314000+00:00', 'creationTime': '2019-02-12T22:36:19.227686+00:00', 'currentNodeCount': 0, 'errors': None, 'modifiedTime': '2019-02-12T22:45:58.932145+00:00', 'nodeStateCounts': {'idleNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0, 'preparingNodeCount': 0, 'runningNodeCount': 0, 'unusableNodeCount': 0}, 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT300S'}, 'targetNodeCount': 0, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


In [4]:
# top k items to recommend
TOP_K = 10

# Select Movielens data size: 100k, 1m, 10m, or 20m
MOVIELENS_DATA_SIZE = '1m'

### 3 Download dataset and upload to data store
Now make the data accessible remotely by uploading that data from your local machine into Azure so it can be accessed for remote training. The datastore is a convenient construct associated with your workspace for you to upload/download data, and interact with it from your remote compute targets. It is backed by Azure blob storage account.

The data files are uploaded into a directory named `data` at the root of the datastore. Here's detailed documentation about [datastore](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-access-data).

In [5]:
DATA_DIR = 'aml_data'
TARGET_DIR = 'movielens'
#os.makedirs('./data', exist_ok = True)

os.makedirs(DATA_DIR, exist_ok=True)
print(MOVIELENS_DATA_SIZE)

# download dataset
data = movielens.load_pandas_df(
    size=MOVIELENS_DATA_SIZE,
    header=['UserId','MovieId','Rating','Timestamp']
)

# upload dataset to workspace datastore
data_file_name = "movielens_" + MOVIELENS_DATA_SIZE + "_data.pkl"
data.to_pickle(os.path.join(DATA_DIR, data_file_name))

ds = ws.get_default_datastore()

ds.upload(src_dir=DATA_DIR, target_path=TARGET_DIR, overwrite=True, show_progress=True)

1m
/data/home/cathack/notebooks/Recommenders/notebooks/00_quick_start
http://files.grouplens.org/datasets/movielens/ml-1m.zip


$AZUREML_DATAREFERENCE_131694da6915425d8f447dd049ddbc39

# 1 Prepare training script
### Create a directory
Create a directory to deliver the necessary code from your computer to the remote resource.

In [6]:
script_folder = './movielens-sar'
os.makedirs(script_folder, exist_ok=True)

### Create a training script
To submit the job to the cluster, first create a training script. Run the following code to create the training script called `train.py` in the directory you just created. This training adds a regularization rate to the training algorithm, so produces a slightly different model than the local version.

In [7]:
%%writefile $script_folder/train.py

import argparse
import os
import numpy as np
import pandas as pd
import itertools
import logging
import time

from azureml.core import Run
from sklearn.externals import joblib

from reco_utils.dataset import movielens
from reco_utils.dataset.python_splitters import python_random_split
from reco_utils.evaluation.python_evaluation import map_at_k, ndcg_at_k, precision_at_k, recall_at_k
from reco_utils.recommender.sar.sar_singlenode import SARSingleNode

TARGET_DIR = 'movielens'
OUTPUT_FILE_NAME = 'outputs/movielens_sar_model.pkl'


# get hold of the current run
run = Run.get_context()

# let user feed in 2 parameters, the location of the data files (from datastore), and the regularization rate of the logistic regression model
parser = argparse.ArgumentParser()
parser.add_argument('--data-folder', type=str, dest='data_folder', help='data folder mounting point')
parser.add_argument('--data-file', type=str, dest='data_file', help='data file name')
parser.add_argument('--top-k', type=int, dest='top_k', default=10, help='top k items to recommend')
parser.add_argument('--data-size', type=str, dest='data_size', default=10, help='Movielens data size: 100k, 1m, 10m, or 20m')
args = parser.parse_args()

data_pickle_path = os.path.join(args.data_folder, args.data_file)

data = pd.read_pickle(path=data_pickle_path)

train, test = python_random_split(data)

# instantiate the SAR algorithm and set the index
header = {
    "col_user": "UserId",
    "col_item": "MovieId",
    "col_rating": "Rating",
    "col_timestamp": "Timestamp",
}

logging.basicConfig(level=logging.DEBUG, 
                    format='%(asctime)s %(levelname)-8s %(message)s')

model = SARSingleNode(
    remove_seen=True, similarity_type="jaccard", 
    time_decay_coefficient=30, time_now=None, timedecay_formula=True, **header
)

# train the SAR model
start_time = time.time()

model.fit(train)

train_time = time.time() - start_time
run.log(name="Training time", value=train_time)

start_time = time.time()

top_k = model.recommend_k_items(test)

test_time = time.time() - start_time
run.log(name="Prediction time", value=test_time)

# TODO: remove this call when the model returns same type as input
top_k['UserId'] = pd.to_numeric(top_k['UserId'])
top_k['MovieId'] = pd.to_numeric(top_k['MovieId'])

# evaluate
eval_map = map_at_k(test, top_k, col_user="UserId", col_item="MovieId", 
                    col_rating="Rating", col_prediction="prediction", 
                    relevancy_method="top_k", k=args.top_k)
eval_ndcg = ndcg_at_k(test, top_k, col_user="UserId", col_item="MovieId", 
                      col_rating="Rating", col_prediction="prediction", 
                      relevancy_method="top_k", k=args.top_k)
eval_precision = precision_at_k(test, top_k, col_user="UserId", col_item="MovieId", 
                                col_rating="Rating", col_prediction="prediction", 
                                relevancy_method="top_k", k=args.top_k)
eval_recall = recall_at_k(test, top_k, col_user="UserId", col_item="MovieId", 
                          col_rating="Rating", col_prediction="prediction", 
                          relevancy_method="top_k", k=args.top_k)

run.log("map", eval_map)
run.log("ndcg", eval_ndcg)
run.log("precision", eval_precision)
run.log("recall", eval_recall)

os.makedirs('outputs', exist_ok=True)
# note file saved in the outputs folder is automatically uploaded into experiment record
joblib.dump(value=model, filename=OUTPUT_FILE_NAME)

Overwriting ./movielens-sar/train.py


In [8]:
shutil.rmtree('./movielens-sar/reco_utils/', ignore_errors=True)
shutil.copytree('../../reco_utils/', './movielens-sar/reco_utils/')

'./movielens-sar/reco_utils/'

# 2 Run training script
### Create an estimator
An estimator object is used to submit the run.  Create your estimator by running the following code to define:

* The name of the estimator object, `est`
* The directory that contains your scripts. All the files in this directory are uploaded into the cluster nodes for execution. 
* The compute target.  In this case you will use the AmlCompute you created
* The training script name, train.py
* Parameters required from the training script 
* Python packages needed for training

In this tutorial, this target is AmlCompute. All files in the script folder are uploaded into the cluster nodes for execution. `ds.as_mount()` mounts a datastore on the remote compute and returns the folder. See documentation [here](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-access-data#access-datastores-during-training).

In [9]:
script_params = {
    '--data-folder': ds.as_mount(),
    '--data-file': 'movielens/' + data_file_name,
    '--top-k': TOP_K,
    '--data-size': MOVIELENS_DATA_SIZE
}

est = Estimator(source_directory=script_folder,
                script_params=script_params,
                compute_target=compute_target,
                entry_script='train.py',
                conda_packages=['pandas'],
                pip_packages=['sklearn'])

### Submit the job to the cluster
Run the experiment by submitting the estimator object. You can check the status of current experiment in Azure Portal by clicking link below.

In [ ]:
# create experiment
EXPERIMENT_NAME = 'movielens-sar'
exp = Experiment(workspace=ws, name=EXPERIMENT_NAME)

run = exp.submit(config=est)
run

You should see simialar output as below
![Experiment submit output](https://docs.microsoft.com/en-us/azure/machine-learning/service/media/quickstart-get-started/view_exp.png)
Azure Portal looks like this
![Azure Portal Experiment](https://docs.microsoft.com/en-us/azure/machine-learning/service/media/quickstart-get-started/web-results.png)


# 3 Monitor remote run

### Jupyter widget

Alternatively, watch the progress of the run with a Jupyter widget.  Like the run submission, the widget is asynchronous and provides live updates every 10-15 seconds until the job completes.

In [11]:
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'NOTSET',…